# BIO294: bioinformatics for comparative and evolutionary genomics
christoph.stritt@botinst.uzh.ch  
*11 November 2020*

# First steps to building your own bioinformatics toolbox with Python

#### Online documentation
https://www.python.org/doc/  
https://biopython.org/  
http://rosalind.info/problems/locations/


## A simple program to explore gene trees 
### Phytochrome A

![Gene tree for Phytochrome B](phyA.png)

# Preliminaries


#### Download the course data from github to your Desktop

```console
cd ~/Desktop
git clone https://github.com/cstritt/bio294

```

In the data folder you'll find a file containing protein sequences of _Brachypodium distachyon_ and some other species, downloaded from Phytozome ( https://phytozome-next.jgi.doe.gov ).

Our goal is to create a tree for the Phytochrome A gene **Bradi1g10520**, showing homologous genes within _B. distachyon_ and in other plant species.

Let's take a look at the input files.


```console
cd data/
ls

zless -S Bdistachyon_314_v3.1.protein_primaryTranscriptOnly.fa.gz

# Unzip all fasta files 
gunzip *.gz

# Is there some information available for the gene? 
grep Bradi1g10520 Bdistachyon_314_v3.1.defline.txt
grep Bradi1g10520 Bdistachyon_314_v3.1.annotation_info.txt
```

#### Program outline
**A)** Fetch fasta entry for the gene of interest  
**B)** Blast fasta entry against a local data base  
**C)** Extract homolog sequences and align them  
**D)** Estimate gene tree  


#### Exercises
In case you are stuck, possible solutions for the exercises are given in the file dirtyTree.py.

## A) Fetch fasta entry for the gene of interest

First, we want to extract the protein sequence of our focal gene and write it to a fasta file, which can then be used with blast.  
This can be done using some bash bricolage, for example:


```console
sed -n -e '/Bradi5g25817/,/^>/ p' Bdistachyon_314_v3.1.protein_primaryTranscriptOnly.fa | sed -e ';$d' 
```

Let's try the same with Python. 


In [ ]:
gene = 'Bradi1g10520'
path_to_gene_space = 'data/Bdistachyon_314_v3.1.protein_primaryTranscriptOnly.fa'

switch = 0
gene_seq = ''
with open(path_to_gene_space) as f:
    for line in f:
        
        if line.startswith('>'):
            switch = 1 if gene in line else 0
            
        if switch == 1:
            gene_seq += line  
            
gene_seq

In [ ]:
# Write to file          
with open(gene + '.fasta', 'w') as g:
    g.write(gene_seq)

In [ ]:
# The same can be done more simply with Biopython
import sys
from Bio import SeqIO

# Display some information about the SeqIO object
# help(SeqIO)

In [ ]:
# Fetch the gene sequence
for seq_record in SeqIO.parse(path_to_gene_space, "fasta"):
    if gene in seq_record.id:
        SeqIO.write(seq_record, sys.stdout, 'fasta')

# Whats the difference between the two outputs?

***Exercise:***  Rewrite the code above as a function, either using the approach with or without Biopython.

## B) Blast fasta entry against a local database

To find homologs, we blast the gene sequence against the gene space, i.e. against the same file from which the gene was extracted. On the command line, this would be
```console
blastp -query Bradi5g25817.fasta -subject Bdistachyon_314_v3.1.protein_primaryTranscriptOnly.fa

```

The Biopython module has a blast interface which allows running blast from within Python. Results are stored in the file defined by the "out" parameter. 

In [ ]:
from Bio.Blast.Applications import NcbiblastpCommandline

#help(NcbiblastxCommandline)
blast_cline = NcbiblastpCommandline(query = gene + ".fasta", 
                                     subject = path_to_gene_space,
                                     outfmt = '6 pident length evalue sseqid', 
                                     out = gene + ".blastout")

# Print command
print(blast_cline)
# Execute command
blast_cline()

***Exercise:*** Take a look at the output file. How many hits does it contain?

### Digression: Blast against the NCBI online database

In [ ]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
  
fasta_string = open(gene + '.fasta').read()
result_handle = NCBIWWW.qblast("blastp", "refseq_protein", fasta_string)

blast_records = NCBIXML.parse(result_handle)
blast_records = list(blast_records)

# Can you find the homolog sequences in the blast_records list?

## C) Extract homolog sequences and align them

***Exercise:***  
Filter the output of the local blast: create a list with the names of the genes with a percentage identity > 30, alignment length > 150, and an e-value < 0.0001 (Panchy et al. 2016). How 'robust' are these thresholds, i.e. how does the number of 'homologs' change when the thresholds change?

In [ ]:
min_perc_id = 30
min_aln_len = 150
max_eval = 0.0001

homologs = []
# Continue here

In [ ]:
# Extract all homolog sequences from the gene space and write them to a file
from Bio import SeqIO

recs = []

for seq_record in SeqIO.parse(path_to_gene_space, "fasta"):
    
    if any(x in seq_record.id for x in homologs):    
        recs.append(seq_record)

SeqIO.write(recs, gene + ".homologs.fasta", 'fasta')

In [ ]:
""" Run the muscle aligner through the Biopython interface
"""
from Bio import AlignIO
from Bio.Align.Applications import MuscleCommandline
#help(MuscleCommandline)

In [ ]:
# Define the command
cline = MuscleCommandline(input=gene + ".homologs.fasta", 
                          out=gene + ".homologs.aligned.fasta")

# Show the command
print(cline)
# Run it
cline()

# Load the alignment
aln = AlignIO.read(gene + '.homologs.aligned.fasta', 'fasta')
print(aln)

In [ ]:
# Access information in the alignment
print(aln[0])

***Exercise:***  
How many gaps are there in each aligned sequence? 

In [ ]:
# Trim alignment
import strumenti
aln_trimmed = strumenti.trim_alignment(aln, max_prop_missing=0)
print(aln_trimmed)

## D) Estimate gene tree
Biopython is great for parsing phylogenetic trees and contains a neighbor-joining algorithm for estimating phylogenetic trees (https://en.wikipedia.org/wiki/Neighbor_joining).

In [ ]:
import sys
from Bio import Phylo

tree = strumenti.neighbor_joining_tree(aln_trimmed, prot_model='blosum62')
tree_out = Phylo.draw_ascii(tree, file=sys.stdout, column_width=50)

## Exercises:
1) Implement the code above as a program which can be run from the command line. (Hint: use the *sys* module to parse command line arguments.)


2) Can you find an easy way to include sequences from other species in the analysis? 

3) Create a tree for the NBS-LRR gene Bradi1g00960, including all species in the data folder. How does its homolog tree differ from Phytochrome A? Why?  


4) Run the program for some more genes: Bradi1g64360, Bradi1g12340, Bradi1g31280. Instead of typing each command individually in the command line, try to use a for loop and write the trees to a file.    

5) Add minimum percentage identity as a parameter to the program.  